In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, two_layer_basic, post_process


import os
from os import listdir, mkdir
from os.path import join, isdir
set_figsize(20, 15)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
device = 'cpu'
output_dir = 'generated_output'
ave_model = average_model()

In [27]:
def generate_evens(model, sequence, post_proc, save_dir):
    saveto = join(output_dir, save_dir)
    if not isdir(saveto):
        mkdir(saveto)
    with torch.no_grad():
        model.eval()
        generated = [post_proc(sequence[0])]
        for i in range(2,len(sequence),2):
            gen = model((sequence[i-2][None,:].to(device=device), sequence[i][None,:].to(device=device)))
            generated += [post_proc(gen),  post_proc(sequence[i].to(device=device))]
        for i, f in enumerate(generated):
            name = join(saveto, '%d.npy'%i)
            np.save(name, f)

In [35]:
def generate_between(model, sequence, post_proc, save_dir):
    saveto = join(output_dir, save_dir)
    if not isdir(saveto):
        mkdir(saveto)
    with torch.no_grad():
        model.eval()
        generated = [post_proc(sequence[0])]
        for i in range(1,len(sequence)):
            gen = model((sequence[i-1][None,:].to(device=device), sequence[i][None,:].to(device=device)))
            generated += [post_proc(torch.squeeze(gen)),  post_proc(sequence[i].to(device=device))]
        for i, f in enumerate(generated):
            name = join(saveto, '%d.npy'%i)
            np.save(name, f)

In [7]:
preproc_type = "pixel"
post_proc = post_process(kind=preproc_type).to(device=device)
val_seq = TAVR_Sequence("__valid", preproc=preproc_type)

In [28]:
generate_evens(ave_model, val_seq[1], post_proc, "ave_evens")

In [17]:
from utils.run_model import load_model
from Models.nm_layer import nm_layer_net, Parallel_Residual
model = Parallel_Residual(2, [4,8],[4,1])
load_model("Model 24 (Parallel Residual) Run 0", 646, model)

model loaded from model_checkpoints/Model 24 (Parallel Residual) Run 0/Model 24 (Parallel Residual) Run 0-646


In [31]:
generate_evens(model, val_seq[1], post_proc, "m24_evens")

In [36]:
generate_between(ave_model, val_seq[1], post_proc, "ave_between")

In [37]:
generate_between(model, val_seq[1], post_proc, "m24_between")